In [1]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy as np
import pandas as pd
import pickle

C:\Users\spark\AppData\Local\Temp\ipykernel_48628\3157163411.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [2]:
window_size = 10
batch_size_exp = 10
epoch_exp = 60
neurons_exp = 5
predict_values_exp = 500
lag_exp=72

In [3]:
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [4]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

In [5]:
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [6]:
def scale(data_norm):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(data_norm)
    # transform train
    data_norm = data_norm.reshape(data_norm.shape[0], data_norm.shape[1])
    data_scaled = scaler.transform(data_norm)
    return scaler, data_scaled

In [7]:
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]
     

In [8]:
from keras.layers import Activation, Dense, BatchNormalization, TimeDistributed
from tensorflow.keras.optimizers import Adam

def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    print(X.shape, y.shape)
    model = Sequential()
    model.add(LSTM(neurons_exp))
    model.add(BatchNormalization())
    model.add(Dense(50))
    model.add(Activation('relu'))
    model.add(Dense(50))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.05))
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()
    return model

In [9]:
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    #print(X)
    yhat = model.predict(X, batch_size=batch_size_exp)
    return yhat[0,0]

In [10]:
df = pd.read_csv("dataset.csv", index_col = "DATE")
df

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,AWND,FMTM,PGTM,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,city,state,region,th_power,power
DATE,,,,,,,,,,,,,,,,,,,,
2010-01-01,USW00014942,"OMAHA EPPLEY AIRFIELD, NE US",41.31186,-95.90186,298.8,3.800000,1311.000000,1159.000000,11.419891,-11.100000,-23.30000,350.000000,330.000000,8.000000,10.300000,Des Moines,IA,Midwest Region,105682.164827,0
2010-01-02,USW00014942,"OMAHA EPPLEY AIRFIELD, NE US",41.31186,-95.90186,298.8,1.700000,222.000000,1950.000000,11.419891,-16.700000,-25.60000,10.000000,30.000000,5.400000,7.200000,Des Moines,IA,Midwest Region,0.000000,0
2010-01-03,USW00014942,"OMAHA EPPLEY AIRFIELD, NE US",41.31186,-95.90186,298.8,2.400000,1547.000000,1558.000000,11.419891,-13.300000,-20.60000,10.000000,20.000000,5.400000,5.800000,Des Moines,IA,Midwest Region,0.000000,0
2010-01-04,USW00014942,"OMAHA EPPLEY AIRFIELD, NE US",41.31186,-95.90186,298.8,2.700000,1606.000000,1605.000000,11.419891,-15.600000,-28.90000,10.000000,10.000000,4.000000,4.500000,Des Moines,IA,Midwest Region,0.000000,0
2010-01-05,USW00014942,"OMAHA EPPLEY AIRFIELD, NE US",41.31186,-95.90186,298.8,1.300000,2219.000000,2221.000000,11.419891,-13.300000,-26.70000,150.000000,140.000000,3.600000,4.000000,Des Moines,IA,Midwest Region,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-29,USW00013881,"CHARLOTTE DOUGLAS AIRPORT, NC US",35.22254,-80.95433,222.6,2.300000,1836.559043,1422.573668,18.900000,22.200000,15.60000,150.000000,150.000000,4.000000,5.800000,Charleston,SC,Coastal Areas,0.000000,0
2023-04-30,USW00013881,"CHARLOTTE DOUGLAS AIRPORT, NC US",35.22254,-80.95433,222.6,3.800000,1836.559043,1422.573668,18.500000,22.200000,10.00000,300.000000,300.000000,12.500000,17.900000,Charleston,SC,Coastal Areas,105682.164827,0
2023-05-01,USW00013881,"CHARLOTTE DOUGLAS AIRPORT, NC US",35.22254,-80.95433,222.6,5.200000,1836.559043,1422.573668,14.100000,20.000000,8.30000,220.000000,260.000000,11.200000,14.800000,Charleston,SC,Coastal Areas,270807.658405,0


In [12]:
for station in df.STATION.unique():    
    temp_df = df[df['STATION'].isin({station})]
    temp = temp_df[['th_power']]
    raw_values = temp.values
    diff_values = difference(raw_values, 1)
    supervised = timeseries_to_supervised(diff_values, lag_exp)
    supervised_values = supervised.values
    scaler,supervised_values = scale(supervised_values)
    train_scaled, test_scaled = supervised_values[0:-predict_values_exp], supervised_values[-predict_values_exp:]
    train_scaled.shape
    lstm_model = fit_lstm(train_scaled, batch_size_exp, epoch_exp, neurons_exp)
    Pkl_Filename = "Power models/LSTM/"+station+".pkl"  
    with open(Pkl_Filename, 'wb') as file:  
        pickle.dump(lstm_model, file)


(4370, 1, 72) (4370,)
437/437 [==============================] - 1s 3ms/step - loss: 0.0156
(4369, 1, 72) (4369,)
437/437 [==============================] - 2s 5ms/step - loss: 0.0267
(9241, 1, 72) (9241,)
925/925 [==============================] - 4s 4ms/step - loss: 0.0195
(4370, 1, 72) (4370,)
437/437 [==============================] - 2s 6ms/step - loss: 0.0122
(9241, 1, 72) (9241,)
925/925 [==============================] - 3s 4ms/step - loss: 0.0091
(9241, 1, 72) (9241,)
925/925 [==============================] - 4s 4ms/step - loss: 0.0102
(4370, 1, 72) (4370,)
437/437 [==============================] - 1s 3ms/step - loss: 0.0362
(4370, 1, 72) (4370,)
437/437 [==============================] - 2s 4ms/step - loss: 0.0104
(4370, 1, 72) (4370,)
437/437 [==============================] - 1s 3ms/step - loss: 0.0195
(4169, 1, 72) (4169,)
417/417 [==============================] - 2s 4ms/step - loss: 0.0091
(4368, 1, 72) (4368,)
437/437 [==============================] - 2s 4ms/step - l

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


(4366, 1, 72) (4366,)
437/437 [==============================] - 2s 4ms/step - loss: nan
(4370, 1, 72) (4370,)
437/437 [==============================] - 1s 1ms/step - loss: 0.0028
(4370, 1, 72) (4370,)
437/437 [==============================] - 1s 2ms/step - loss: 0.0097


In [ ]:
# temp.shape

In [ ]:
# raw_values = temp.values
# diff_values = difference(raw_values, 1)

In [ ]:
# supervised = timeseries_to_supervised(diff_values, lag_exp)
# supervised_values = supervised.values

In [ ]:
# scaler,supervised_values = scale(supervised_values)
# train_scaled, test_scaled = supervised_values[0:-predict_values_exp], supervised_values[-predict_values_exp:]
# train_scaled.shape

In [ ]:
# lstm_model = fit_lstm(train_scaled, batch_size_exp, epoch_exp, neurons_exp)

In [ ]:
# predictions = list()
# expectations = list()
# test_pred = list()
# for i in range(len(test_scaled)-window_size):
#     # make one-step forecast
#     X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
#     yhat = forecast_lstm(lstm_model, batch_size_exp, X)#batch_size_exp to 1
#     '''# Start Debug prints
#     print("X: %", X)
#     print("yhat: %", yhat)
#     # End Debug prints'''
#     # Replacing value in test scaled with the predicted value.
#     test_pred = [yhat] + test_pred 
#     if len(test_pred) > lag_exp+1:
#         test_pred = test_pred[:-1]
#     if i+1> lag_exp+1:
#             test_scaled[i+1] = test_pred
#     else:
#         test_scaled[i+1] = np.concatenate((test_pred, test_scaled[i+1, i+1:]),axis=0)

#     # invert scaling
#     yhat = invert_scale(scaler, X, yhat)
#     # invert differencing
#     yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
#     # store forecast
#     expected = raw_values[len(train_scaled) + i + 1]
#     if expected != 0:
#         predictions.append(yhat)
#         expectations.append(expected)
#     print('Day=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

In [ ]:
# expectations = np.array(expectations)
# predictions = np.array(predictions)
# print("Mean Absolute Percent Error: ", (np.mean(np.abs((expectations - predictions) / expectations.mean()))*100))
     


In [ ]:
# fig = pyplot.figure(figsize=(20,8))
# pyplot.plot(expectations, label="True")
# pyplot.plot(predictions, label="Predicted")
# pyplot.legend(loc='upper right')
# pyplot.xlabel("Number of days")
# pyplot.ylabel("wind speed")
# pyplot.show()

In [ ]:
# import pickle 
# Pkl_Filename = "Pickle_RL1_Model.pkl"  

# with open(Pkl_Filename, 'wb') as file:  
#     pickle.dump(lstm_model, file)
    
# with open(Pkl_Filename, 'rb') as file:  
#     Pickled_LR1_Model = pickle.load(file)

# Pickled_LR1_Model

In [ ]:
# score = Pickled_LR1_Model.score(train_scaled, test_scaled)  
# # Print the Score
# print("Test score: {0:.2f} %".format(100 * score))  

# # Predict the Labels using the reloaded Model
# Ypredict = Pickled_LR_Model.predict(test)  

# Ypredict

In [ ]:
# X_t, y_t = train_scaled[:, 0:-1], train_scaled[:, -1]
# X_t = X_t.reshape(X_t.shape[0], 1, X_t.shape[1])
# result = Pickled_LR1_Model.evaluate(X_t , y_t)
# result